# Data Loading Pipeline

1. Tokenize the text
2. Sliding window approach to create sequences
3. Load to `DataLoader` class
4. Create embeddings layer
5. Create positional embeddings layer
6. Add positional embeddings to the embeddings

In [1]:
import tiktoken
from tiktoken.core import Encoding
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDataset(Dataset):
    def __init__(self, text: str, tokenizer: Encoding, max_length: int, step: int):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, step):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [2]:
def create_dataloader_v1(
        text: str,
        batch_size: int,
        max_length: int,
        step: int,
        shuffle: bool = True,
        drop_last: bool = True,
        num_workers: int = 0,
):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(text, tokenizer, max_length, step)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
    )
    return dataloader

In [3]:
with open("../../data/the_verdict.txt", "r") as f:
    raw_text = f.read()

max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, step=4, shuffle=False)

data_iter = iter(dataloader)
input_ids, target_ids = next(data_iter)
print(input_ids)
print(target_ids)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [4]:
vocab_size = 50257
output_dim = 256
context_length = 1024

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
position_embedding_layer = torch.nn.Embedding(context_length, output_dim)

token_embeddings = token_embedding_layer(input_ids)
position_embeddings = position_embedding_layer(torch.arange(max_length))


In [8]:
for batch in dataloader:
    x, y  = batch

    token_embeddings = token_embedding_layer(x)
    print(token_embeddings)
    position_embeddings = position_embedding_layer(torch.arange(max_length))
    print(position_embeddings)
    input_embeddings = token_embeddings + position_embeddings

    break

tensor([[[-1.9669, -1.2198, -0.6559,  ...,  0.5926,  0.7129, -1.3824],
         [ 0.9595,  0.0899,  0.4915,  ..., -0.2249, -0.1964,  0.6096],
         [-0.7359, -0.0094, -0.7402,  ...,  0.0481, -0.2016,  0.1673],
         [-0.6303, -0.6080,  0.7900,  ...,  0.4257,  0.7381,  0.6094]],

        [[-0.4139, -0.2746, -0.3565,  ..., -1.1815,  0.9983,  1.1216],
         [-0.3686, -1.6352, -0.3563,  ..., -0.1397,  0.0161,  0.7463],
         [ 0.1056,  0.8492, -1.3161,  ...,  0.6897,  1.2767, -0.2068],
         [ 0.0859, -0.1970, -2.0486,  ..., -0.4712, -1.0559,  0.5522]],

        [[-0.5986,  0.0024, -2.0352,  ..., -0.0214,  0.2633, -0.3674],
         [ 0.5695, -0.4940, -0.1517,  ...,  0.2831, -1.2831, -1.5886],
         [-1.9219,  2.1224,  0.2398,  ..., -0.8620, -0.5231,  1.4595],
         [ 0.9376,  0.9730,  1.2529,  ...,  0.3224, -0.0204, -0.0700]],

        ...,

        [[ 1.5977, -1.6930,  1.4751,  ..., -0.8977, -0.2904,  0.0308],
         [ 1.4394, -1.6410,  0.1755,  ..., -1.4297,  0.86

In [6]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])
